# Never a sity so real


In [1]:
import env
import utils

# We use Gemini directly instead through langchain
# because of safety settings,
# langchain is developing this and not hasn't released yet
import google.generativeai as genai
import os

genai.configure(
    api_key=os.environ.get("GOOGLE_API_KEY")
)
llm_gemini = genai.GenerativeModel(
    model_name = "gemini-pro",
    safety_settings=utils.safety_config
)


In [8]:
file_name = "Never a City So Real A Walk in Chicago (Alex Kotlowitz) (Z-Library)"
input_file = f"Chicago/{file_name}.pdf"
output_file = f"Chicago/{file_name}-cn.txt"

In [9]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader(input_file)
pages = loader.load_and_split()

In [11]:
file_output = open(output_file, "a")

for index, page in enumerate(pages[:]):
    prompt = f""" Suppose you are a translator, and your target language is simplified Chinese, 
    Your audiences are Chinese native young students in Junior school about 13 years old. 
    Translate the following english text to simplified Chinese: \ ```{page.page_content}``` """ 
    
    response = llm_gemini.generate_content(
        prompt,
        safety_settings=utils.safety_config
    )

    file_output.write(f"\n---- page: {index} ---- \n")
    if len(response.candidates) > 0:
        # we only get the first candidate
        for p in response.candidates[0].content.parts:
            file_output.write(p.text.strip())
        file_output.flush()
    
    # if len(response.parts) > 0:
    #     for p in response.parts:
    #         file_output.write(p.text.strip())
    #     file_output.flush()

file_output.close()

In [6]:
# test
output_test_file = f"Chicago/test-cn.txt"
output_test = open(output_test_file, "a")
for index, page in enumerate(pages[119:123]):
    prompt = f""" Suppose you are a translator, and your target language is simplified Chinese, 
    Your audiences are Chinese native young students in Junior school about 13 years old. 
    Translate the following english text to simplified Chinese: \ ```{page.page_content}``` """ 
    
    response = llm_gemini.generate_content(
        prompt,
        safety_settings=utils.safety_config
    )

    for candidate in response.candidates:
        [part.text for part in candidate.content.parts]

    print(f"----{index+119}----")
    if len(response.candidates) > 0:
        print(f"----candid----")
        # output_test.write("--candidates--")
        # print(response.candidates)
        for c in response.candidates:
            for p in c.content.parts:
                output_test.write(p.text.strip())
            output_test.flush()
    # if len(response.parts) > 0:
    #     print(f"----normal----")
    #     output_test.write("--text--")
    #     for p in response.parts:
    #         output_test.write(p.text.strip())
    #         output_test.flush()

output_test.close()

InternalServerError: 500 An internal error has occurred. Please retry or report in https://developers.generativeai.google/guide/troubleshooting